In [ ]:
%pip install --upgrade pip setuptools wheel

In [1]:
%pip install pycaret

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
from pycaret.time_series import TSForecastingExperiment
import pandas as pd
import numpy as np
import matplotlib

In [3]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
def pycaret(dataset, test_size, target):

    dataset.index = range(0, dataset.shape[0])
    train, test = dataset.head(int(dataset.shape[0]-test_size)), dataset.tail(test_size)
    exp_auto = TSForecastingExperiment()
    exp_auto.setup(
        data=train, target=target,
        enforce_exogenous=True,
        numeric_imputation_target="ffill", numeric_imputation_exogenous="ffill",
        session_id=42, 
    )

    real = test[target]
    
    best = exp_auto.compare_models(verbose=True)
    print(best)
    best_model = exp_auto.tune_model(
                        best,
                        choose_better=True,
                        n_iter=50,
                        fold=3,
                        search_algorithm="random",
                        tuner_verbose=True,
                    )
    
    forecast = exp_auto.predict_model(best_model, fh=30)

    return real.values, forecast.values.flatten()

In [5]:
# Example: Create a simple time series dataset
# Your dataset should have a column for the target variable and optionally exogenous variables
data = pd.DataFrame({
    'date': pd.date_range(start='2020-01-01', periods=100, freq='D'),
    'sales': [i + (i * 0.1) + np.random.normal(0, 2) for i in range(100)],  # target
    'promotion': [1 if i % 7 == 0 else 0 for i in range(100)]  # exogenous variable
})

# Set date as index if you have a datetime index (though the function uses range index)
# data = data.set_index('date')

# Call the function
real_values, forecast_values = pycaret(
    dataset=data,        # Your DataFrame
    test_size=30,        # Number of points to use for testing (last 30 points)
    target='sales'       # Name of the column to forecast
)

print("Real values:", real_values)
print("Forecast values:", forecast_values)

,Description,Value
0,session_id,42
1,Target,sales
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(70, 3)"
5,Transformed data shape,"(70, 3)"
6,Transformed train set shape,"(69, 3)"
7,Transformed test set shape,"(1, 3)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


,Model,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE,TT (Sec)
croston,Croston,4.2569,3.3771,10.2434,10.2434,0.1398,0.1505,0.3133


Croston()


,cutoff,MASE,RMSSE,MAE,RMSE,MAPE,SMAPE
0,65.0000,1.0422,0.8258,2.5168,2.5168,0.0346,0.0352
1,66.0000,0.3592,0.2847,0.8631,0.8631,0.0122,0.0122
2,67.0000,1.9019,1.5113,4.5655,4.5655,0.0604,0.0623
Mean,nan,1.1011,0.8739,2.6485,2.6485,0.0357,0.0365
SD,nan,0.6312,0.5019,1.5144,1.5144,0.0197,0.0205


Fitting 3 folds for each of 50 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 10 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 tasks      | elapsed:    0.7s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:    1.3s finished


Real values: [ 76.70977408  76.66133882  78.32904649  80.6345217   80.91347264
  85.90774362  85.74733061  86.55450448  87.23128269  83.04908471
  89.62482644  89.97538367  90.14497425  91.01471755  90.98889098
  93.49069157  93.55700878  94.20935848  98.10407432  97.56700597
  98.23796528 104.1398142  102.39544006 104.03748955 107.55829718
 103.48115011 102.22973871 105.8506914  105.5116558  107.22546915]
Forecast values: [73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669
 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669
 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669 73.1669
 73.1669 73.1669 73.1669]
